<a href="https://colab.research.google.com/github/ebi19912/AI/blob/main/API_model_with_six_models_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('/content/drive/My Drive/COMOFOD1/train',target_size=(224, 224),batch_size=32,class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory('/content/drive/My Drive/COMOFOD1/val',target_size=(224, 224),batch_size=32,class_mode='categorical')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('/content/drive/My Drive/COMOFOD1/test',target_size=(224, 224),batch_size=32,class_mode='categorical')

Found 7004 images belonging to 2 classes.
Found 2009 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen1 = ImageDataGenerator(rescale=1./255)
validation_datagen1 = ImageDataGenerator(rescale=1./255)
train_generator1 = train_datagen1.flow_from_directory('/content/drive/My Drive/COMOFOD1/train',target_size=(299, 299),batch_size=32,class_mode='categorical')
validation_generator1 = validation_datagen1.flow_from_directory('/content/drive/My Drive/COMOFOD1/val',target_size=(299, 299),batch_size=32,class_mode='categorical')
test_datagen1 = ImageDataGenerator(rescale=1./255)
test_generator1 = test_datagen1.flow_from_directory('/content/drive/My Drive/COMOFOD1/test',target_size=(299, 299),batch_size=32,class_mode='categorical')

Found 7004 images belonging to 2 classes.
Found 2009 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
print(validation_generator.image_shape)
print(validation_generator1.image_shape)

(224, 224, 3)
(299, 299, 3)


In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense,Flatten,UpSampling2D,Reshape
from tensorflow.keras.applications import vgg16,vgg19,ResNet50,InceptionV3,InceptionResNetV2
from keras.applications.densenet import DenseNet201,DenseNet121
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.applications.mobilenet import MobileNet
import scipy
import os
import cv2
from PIL import Image
import numpy as np
#from keras.applications.resnet50 import ResNet50, preprocess_input

In [ ]:
vgg=vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))
vgg.summary()
for layer in vgg.layers[:-5]:
  print(layer.name)
  layer.trainable=False
x=vgg.layers[-1].output
print(x.shape)

In [ ]:
#vgg16
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_vgg16.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
#output1 = layers.concatenate([z1], axis=3,name='myconcate')
output1=GlobalAveragePooling2D()(x)
output1=Dense(2,activation="softmax")(output1)

modelvgg=Model([vgg.input],output1)
modelvgg.compile(loss=['categorical_crossentropy'],optimizer=Adam(learning_rate=1e-5),metrics=["acc"])
print(modelvgg.summary())

In [ ]:
#vgg16
history = modelvgg.fit(
train_generator,
steps_per_epoch=219,
epochs=50,
validation_data=validation_generator,validation_steps=63,callbacks=[es, mc])

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
print(loss)
print("***************************")
print(val_loss)

In [ ]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
print(acc)
print("*********************************")
print(val_acc)

In [ ]:
################################
import tensorflow as tf
dense=DenseNet201(weights='imagenet',include_top=False,input_shape=(224,224,3))
dense.summary()
#dense._name = "dense"
for layer in dense.layers[:-5]:
  print(layer.name)
  layer.trainable=False
y=dense.layers[-1].output
print(y.shape)

In [ ]:
#denseNet201
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_densenet201.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
#output1 = layers.concatenate([z1], axis=3,name='myconcate')
output1=GlobalAveragePooling2D()(y)
output1=Dense(2,activation="softmax")(output1)

modeldense=Model([dense.input],output1)
modeldense.compile(loss=['categorical_crossentropy'],optimizer=Adam(lr=1e-5),metrics=["acc"])
print(modeldense.summary())

In [ ]:
#densenet201
history = modeldense.fit(
train_generator,
steps_per_epoch=219,
epochs=50,
validation_data=validation_generator,validation_steps=63,callbacks=[es, mc])

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
print(loss)
print("***************************")
print(val_loss)

In [ ]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
print(acc)
print("*********************************")
print(val_acc)

In [ ]:
################################
import tensorflow as tf
resnet=ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
resnet.summary()
for layer in resnet.layers[:-5]:
  #model._name = "new_name"
  layer._name = "res"+layer.name
  print(layer.name)
  #layer._name = "res"+layer.name
  layer.trainable=False
z=resnet.layers[-1].output
print(z.shape)

In [ ]:
#resnet50
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_resnet.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
#output1 = layers.concatenate([z1], axis=3,name='myconcate')
output1=GlobalAveragePooling2D()(z)
output1=Dense(2,activation="softmax")(output1)

modelresnet=Model([resnet.input],output1)
modelresnet.compile(loss=['categorical_crossentropy'],optimizer=Adam(lr=1e-5),metrics=["acc"])
print(modelresnet.summary())

In [ ]:
#resnet50
history = modelresnet.fit(
train_generator,
steps_per_epoch=219,
epochs=50,
validation_data=validation_generator,validation_steps=63,callbacks=[es, mc])

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
print(loss)
print("***************************")
print(val_loss)

In [ ]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
print(acc)
print("*********************************")
print(val_acc)

In [ ]:
import tensorflow as tf
import numpy as np
resnetinc=InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(299,299,3))
resnetinc.summary()
for layer in resnetinc.layers[:-5]:
  #print(layer.name)
  layer.trainable=False
u=resnetinc.layers[-1].output
#x=Reshape((16,4,2048))(x)
u= tf.image.resize(u, [7,7])

print(u.shape)

In [ ]:
#InceptionResNetV2
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_resnetinc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
#output1 = layers.concatenate([z1], axis=3,name='myconcate')
output1=GlobalAveragePooling2D()(u)
output1=Dense(2,activation="softmax")(output1)

modelresnetinc=Model([resnetinc.input],output1)
modelresnetinc.compile(loss=['categorical_crossentropy'],optimizer=Adam(lr=1e-5),metrics=["acc"])
print(modelresnetinc.summary())

In [ ]:
#InceptionResNetV2
history = modelresnetinc.fit(
train_generator1,
steps_per_epoch=219,
epochs=50,
validation_data=validation_generator1,validation_steps=63,callbacks=[es, mc])

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
print(loss)
print("***************************")
print(val_loss)

In [ ]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
print(acc)
print("*********************************")
print(val_acc)

In [ ]:
import tensorflow as tf
import numpy as np
inc=InceptionV3(weights='imagenet',include_top=False,input_shape=(299,299,3))
inc.summary()
for layer in inc.layers[:-5]:
  #print(layer.name)
  layer.trainable=False
v=inc.layers[-1].output
#x=Reshape((16,4,2048))(x)
v= tf.image.resize(v, [7,7])

print(v.shape)

In [ ]:
#InceptionV3
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_inc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
#output1 = layers.concatenate([z1], axis=3,name='myconcate')
output1=GlobalAveragePooling2D()(v)
output1=Dense(2,activation="softmax")(output1)

modelinc=Model([inc.input],output1)
modelinc.compile(loss=['categorical_crossentropy'],optimizer=Adam(lr=1e-5),metrics=["acc"])
print(modelinc.summary())

In [ ]:
#InceptionV3
history = modelinc.fit(
train_generator1,
steps_per_epoch=219,
epochs=50,
validation_data=validation_generator1,validation_steps=63,callbacks=[es, mc])

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
print(loss)
print("***************************")
print(val_loss)

In [ ]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
print(acc)
print("*********************************")
print(val_acc)

In [ ]:
import tensorflow as tf
mob = tf.keras.applications.MobileNet(
    weights="imagenet",  # load weights pretrained on the ImageNet
    input_shape=(224, 224, 3),
    include_top=False  # do not include the ImageNet classifier at the top
)
mob.summary()
for layer in mob.layers[:-5]:
  print(layer.name)
  layer.trainable=False
t=mob.layers[-1].output
print(t.shape)

In [ ]:
#mobilenet
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_mobilenet.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
#output1 = layers.concatenate([z1], axis=3,name='myconcate')
output1=GlobalAveragePooling2D()(t)
output1=Dense(2,activation="softmax")(output1)

modelmob=Model([mob.input],output1)
modelmob.compile(loss=['categorical_crossentropy'],optimizer=Adam(lr=1e-5),metrics=["acc"])
print(modelmob.summary())

In [ ]:
#mobilenet
history = modelmob.fit(
train_generator,
steps_per_epoch=219,
epochs=50,
validation_data=validation_generator,validation_steps=63,callbacks=[es, mc])

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
print(loss)
print("***************************")
print(val_loss)

In [ ]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
print(acc)
print("*********************************")
print(val_acc)

In [ ]:
###########################################   API with 6 models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('/content/drive/My Drive/best_sixmodel.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
output = layers.concatenate([x,y,z,t,u,v], axis=3,name='myconcate')
output=GlobalAveragePooling2D()(output)
output=Dense(2,activation="softmax")(output)

model=Model([vgg.input,dense.input,resnet.input,mob.input,resnetinc.input,inc.input],output)
model.compile(loss=['categorical_crossentropy'],optimizer=Adam(learning_rate=1e-5),metrics=["acc"])
#print(model.summary())

In [ ]:
print(model.summary())

In [ ]:
def combine_data_generator(X1):
    while True:
        x1i=X1.next()
        yield ([x1i[0],x1i[0],x1i[0],x1i[0],x1i[0],x1i[0]],x1i[1])

com=combine_data_generator(train_generator)
print(com)
com1=combine_data_generator(validation_generator)

<generator object combine_data_generator at 0x7f8d6cd23eb0>


In [ ]:
###########merging 6 model 10-5
history = model.fit(
com,
steps_per_epoch=219,
epochs=50,
validation_data=com1,validation_steps=63,callbacks=[es, mc])

In [ ]:
model.save('/content/drive/My Drive/finalsixmodel.h5')

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training_loss')
plt.plot(epochs,val_loss,'r',label='Validation_loss')
plt.title('traing and validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
print(loss)
print("***************************")
print(val_loss)

In [ ]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs=range(1,len(loss)+1)
plt.plot(epochs,acc,'y',label='Training_acc')
plt.plot(epochs,val_acc,'r',label='Validation_acc')
plt.title('traing and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
print(acc)
print("*********************************")
print(val_acc)

In [ ]:
from tensorflow.keras.models import load_model

# load model
model = load_model('/content/drive/My Drive/finalsixmodel.h5')

In [ ]:
################################################## testing on one image for pixel forgery detection
test1_datagen1 = ImageDataGenerator(rescale=1./255)
test1_generator1 = test1_datagen1.flow_from_directory('/content/drive/My Drive/COMOFOD1/test1',target_size=(224, 224),batch_size=32,class_mode='categorical')
test1_datagen2 = ImageDataGenerator(rescale=1./255)
test1_generator2 = test1_datagen2.flow_from_directory('/content/drive/My Drive/COMOFOD1/test1',target_size=(299, 299),batch_size=32,class_mode='categorical')



Found 1 images belonging to 2 classes.
Found 1 images belonging to 2 classes.


In [ ]:
com22=combine_data_generator(test1_generator1)

In [ ]:
pred=model.predict(com22,steps =1)
print(pred[0])

1/1 [==============================] - 2s 2s/step
[0.00343231 0.9965677 ]
